# Notebook used in Lochner et al. 2022

This is the original notebook used in the paper. It's redundant now because we're working from the MAF metrics bundles and I've cleaned up the code a lot. Please use `plots-comparison-v2.ipynb` instead.

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import read_metrics
import plot_tools
from importlib import reload

from scipy.interpolate import griddata

%matplotlib widget

# Defining functions

In [55]:
def read_and_drop(fl, drop=True, dropna_rows=False):
    df = pd.read_csv(fl, index_col=0)
    if drop:
        if 'Problematic' in df.columns:
            df = df[df.Problematic.isna()]
            df = df.drop('Problematic', axis=1)
        if 'group' in df.columns:
            df = df.drop('group', axis=1)
        if 'dbname' in df.index:
            df = df.drop('dbname')

        for c in df.columns:
            df[c] = df[c].astype('float')
            if np.all(np.isnan(df[c].values)):
                df = df.drop(c, axis=1)
        # I have a situation where one metric is missing a couple of runs, but I don't actually need it so 
        # don't want to drop all the runs so this is now a toggle you can switch
        if dropna_rows:
            for i in df.index:
                if np.any(np.isnan(df.loc[i])):
                    df = df.drop(i)
    
    return df

def drop_db_suffix(df):
    inds = []
    for i in list(df.index):
        if '.db' in i:
            inds.append(i[:-3])
        else:
            inds.append(i)
    df.index = inds

def set_plot_type(plot_type='paper'):

    save_plot = True

    if plot_type == 'paper':
        plt.rcParams.update({'font.family':'serif', 'font.size':10, 'figure.dpi':100, #meaningless for pdf only screen
                             'text.usetex':True,
                            'savefig.format':'pdf', 'figure.figsize':figsize_fullwidth})
        fig_dir = 'comparison_plots/'

    elif plot_type == 'presentation':
        fig_dir = 'comparison_plots_presentation/'
        plt.rcParams.update({'font.family':'sans-serif', 'font.size':10, 'figure.dpi':300, 'text.usetex':True,
                        'savefig.format':'png', 'figure.figsize':figsize_fullwidth})

    else:
        fig_dir = ''
        plt.rcParams.update({'font.family':'sans-serif', 'font.size':10, 'figure.dpi':100, 'text.usetex':True,
                        'savefig.format':'png'})
        save_plot = False
    return fig_dir, save_plot

def add_maf_metrics(misc_metrics):
    

    to_include = ['fOArea fO All visits HealpixSlicer',
                  'Median Filter Changes Per Night OneDSlicer',
                  'Mean airmass all bands',
                  'Mean CoaddM5 WFD i band HealpixSlicer',
                  'Median fiveSigmaDepth all bands',
                  'Median fiveSigmaDepth WFD i band',
                  'Median HA all bands',
                  'Median Max Inter-Night Gap all bands HealpixSlicer',
                  'Median Median Inter-Night Gap all bands HealpixSlicer',
                  'Median Inter-Night Gap all bands HealpixSlicer',
                  'Median Median Inter-Night Gap y band HealpixSlicer',
                  'Nvisits WFD',
                  'fONv MedianNvis fO All visits HealpixSlicer',
                  'Rms Median Inter-Night Gap WFD all bands HealpixSlicer',
                 'Median Median Season Length all bands HealpixSlicer']
    maf_metrics_selected = pd.concat((maf_metrics5[to_include], maf_metrics6[to_include], maf_metrics7[to_include]))
    
    
    misc_metrics = pd.merge(misc_metrics, maf_metrics_selected, how='inner', left_index=True, right_index=True)
    return misc_metrics

# Settings and load metrics

In [56]:
reload(read_metrics)
reload(plot_tools)

maf_metrics5 = pd.read_csv('opsim_all_maf_1.%d.csv' %5, index_col=0)
maf_metrics6 = pd.read_csv('opsim_all_maf_1.%d.csv' %6, index_col=0)
maf_metrics7 = pd.read_csv('opsim_all_maf_1.%d.csv' %7, index_col=0)
    
latex_dpi = 72.27
textwidth = 469.75502/latex_dpi
textheight = 650.43001/latex_dpi

figsize_square = (textwidth/2, textwidth/2)
figsize_fullwidth = (textwidth, textwidth/2)
figsize_panels = (textwidth, 0.75*textheight/3)
    
fig_dir, save_plot = set_plot_type('paper')
overwrite_table_db = True

In [57]:
paper_figures = {'area':'\\ref{fig:comparison_area}', 
                 'depth':'\\ref{fig:comparison_depth}', 
                 'filter_dist':'\\ref{fig:comparison_filter_dist}',
                 'internight_gap': '\\ref{fig:comparison_cadence}', 
                 'samefilt':'\\ref{fig:comparison_samefilt}', 
                 'season':'\\ref{fig:comparison_season}', 
                 'weather':'\\ref{fig:comparison_weather}', 
                 'internight_gap_per_filter':'\\ref{fig:internight_gap_filters}', 
                 'mega_metric':'\\ref{fig:mega_metric}',
                 'nexp': '\\ref{fig:comparison_nexp}'}

In [72]:
# Read in all the files
fl = 'metrics_v1_5.csv'
metrics_ori_1_5 = read_and_drop(fl, dropna_rows=False)
misc_metrics_1_5 = read_and_drop('misc-metrics-v1_5.csv', dropna_rows=False)
metrics_rel_1_5, metrics_display_names_1_5, baseline_values_1_5 = read_metrics.read_metrics(fl, 
                                                                                            baseline='baseline_v1.5_10yrs', 
                                                                                            dropna=False)
misc_metrics_1_5 = misc_metrics_1_5.loc[metrics_ori_1_5.index]

fl = 'metrics_v1_6.csv'
metrics_ori_1_6 = read_and_drop(fl, dropna_rows=True)
misc_metrics_1_6 = read_and_drop('misc-metrics-v1_6.csv', dropna_rows=True)
metrics_rel_1_6, metrics_display_names_1_6, baseline_values_1_6 = read_metrics.read_metrics(fl, 
                                                                                            baseline='baseline_nexp1_v1.6_10yrs.db', 
                                                                                            dropna=False)
misc_metrics_1_6 = misc_metrics_1_6.loc[metrics_ori_1_6.index]
drop_db_suffix(metrics_ori_1_6)
drop_db_suffix(misc_metrics_1_6)
drop_db_suffix(metrics_rel_1_6)

fl = 'metrics_outstanding.csv'
metrics_ori_1_7 = read_and_drop('metrics_outstanding.csv', drop=False)
misc_metrics_1_7 = read_and_drop('misc-metrics-v1_7.csv')
metrics_rel_1_7, metrics_display_names_1_7, baseline_values_1_7 = read_metrics.read_metrics('metrics_outstanding.csv', 
                                                                                            baseline='baseline_nexp2_v1.7_10yrs.db', 
                                                                                            dropna=False)

drop_db_suffix(metrics_ori_1_7)
drop_db_suffix(misc_metrics_1_7)
drop_db_suffix(metrics_rel_1_7)
metrics_display_names = metrics_display_names_1_5

#Used to decide which runs to drop
sims_eval1_5 = pd.read_excel('sims_evaluation.xlsx', sheet_name='v1.5', index_col=0)
sims_eval1_6 = pd.read_excel('sims_evaluation.xlsx', sheet_name='v1.6', index_col=0)

# Used here only to start creating a db of files used to create the plots
metrics1_4 = pd.read_csv('metrics_v1_4.csv', index_col=0)
weather_runs = [i for i in metrics1_4.index if 'weather' in i]
metrics1_4['Include'] = [1]*len(metrics1_4)
metrics1_4 = metrics1_4.loc[weather_runs,['Include']]
drop_db_suffix(metrics1_4)

# Similar use
metrics1_7 = pd.read_csv('metrics_outstanding.csv', index_col=0)
baseline_7 = [i for i in metrics1_7.index if 'baseline_nexp' in i]
metrics1_7['Include'] = [1]*len(metrics1_7)
metrics1_7 = metrics1_7.loc[baseline_7,['Include']]
drop_db_suffix(metrics1_7)

# Only runs if we want to overwrite the table of sims used to create plots
if overwrite_table_db:
    all_sims_df = pd.concat((metrics1_4, sims_eval1_5, sims_eval1_6, metrics1_7))
    all_sims_df['Include'] = all_sims_df['Include'].fillna(1)
    all_sims_df = all_sims_df.drop(['Unrealistic', 'Notes'], axis=1)

    for k in paper_figures.keys():
        all_sims_df[k] = [0]*len(all_sims_df)

In [73]:
def select_simulations(combine_1_6 = False, include_unrealistic = True, include_outstanding = True,
                       remove_nexp2=True):
    metrics_original = metrics_ori_1_5.copy()
    misc_metrics = misc_metrics_1_5.copy()
    metrics_rel = metrics_rel_1_5.copy()
    sims_eval = sims_eval1_5
    
    if combine_1_6:
        # *** Included in previous version but I don't think this metric is used anyway ***
#         metrics_original_6 = metrics_original_6.drop(metrics_original_6.columns[-1], axis=1)
        if remove_nexp2:
            inds = []
            for i in metrics_rel_1_6.index:
                if 'nexp2' not in i:
                    inds.append(i)
        else:
            inds = metrics_rel_1_6.index
        misc_metrics = pd.concat((misc_metrics, misc_metrics_1_6.loc[inds]))
        metrics_original = pd.concat((metrics_original, metrics_ori_1_6.loc[inds]))
        metrics_rel = pd.concat((metrics_rel, metrics_rel_1_6.loc[inds]))
        sims_eval = pd.concat((sims_eval1_5, sims_eval1_6))
    
    if include_outstanding:
        metrics_rel = pd.concat((metrics_rel, metrics_rel_1_7.loc[['baseline_nexp1_v1.7_10yrs', 
                                                                   'footprint_8_v1.710yrs']]))
        metrics_original = pd.concat((metrics_original, metrics_ori_1_7.loc[['baseline_nexp1_v1.7_10yrs',
                                                                             'baseline_nexp2_v1.7_10yrs',
                                                                             'footprint_8_v1.710yrs']]))
        misc_metrics = pd.concat((misc_metrics, misc_metrics_1_7.loc[['baseline_nexp1_v1.7_10yrs',
                                                                      'baseline_nexp2_v1.7_10yrs',
                                                                      'footprint_8_v1.710yrs']]))
        
#         metrics_rel = pd.concat((metrics_rel, metrics_rel_1_7.loc[['baseline_nexp1_v1.7_10yrs.db', 'footprint_8_v1.710yrs.db']]))
#         metrics_original = pd.concat((metrics_original, metrics_ori_1_7.loc[['baseline_nexp1_v1.7_10yrs.db','baseline_nexp2_v1.7_10yrs.db','footprint_8_v1.710yrs.db']]))
#         misc_metrics = pd.concat((misc_metrics, misc_metrics_1_7.loc[['baseline_nexp1_v1.7_10yrs.db','baseline_nexp2_v1.7_10yrs.db','footprint_8_v1.710yrs.db']]))
        
    runs_to_drop = list(sims_eval[sims_eval.Include==0].index)
    runs_to_drop += ['daily_ddf_v1.5_10yrs'] # This has disappeared
    runs_to_drop += ['dcr_nham2_ugri_v1.5_10yrs']

    if not include_unrealistic:
        runs_to_drop += list(sims_eval[sims_eval.Unrealistic==1].index)

    inds = []

    for i in metrics_rel.index:
        for r in runs_to_drop:
            if r in i:
                inds.append(i)
    dropped_rel = metrics_rel.drop(inds)

    inds = []
    for i in misc_metrics.index:
        for r in runs_to_drop:
            if r in i:
                inds.append(i)

    misc_metrics = misc_metrics.drop(inds)
    misc_metrics = add_maf_metrics(misc_metrics)
    
    return dropped_rel, metrics_original, misc_metrics

# Area

In [74]:
reload(plot_tools)

<module 'plot_tools' from '/home/michelle/Project/OSTF/journal_plots/plot_tools.py'>

In [79]:
dropped_rel['pz_y10_zlow_stdd'].sort_values()

footprint_newAv1.5_10yrs               -0.195039
wfd_depth_scale0.65_v1.5_10yrs         -0.179779
footprint_big_wfdv1.5_10yrs            -0.147929
footprint_newBv1.5_10yrs               -0.131304
wfd_depth_scale0.70_v1.5_10yrs         -0.114187
baseline_2snaps_v1.5_10yrs             -0.114187
wfd_depth_scale0.70_noddf_v1.5_10yrs   -0.096559
wfd_depth_scale0.75_noddf_v1.5_10yrs   -0.096559
wfd_depth_scale0.75_v1.5_10yrs         -0.096559
bulges_bs_v1.5_10yrs                   -0.096559
bulges_i_heavy_v1.5_10yrs              -0.096559
bulges_cadence_i_heavy_v1.5_10yrs      -0.096559
bulges_cadence_bulge_wfd_v1.5_10yrs    -0.096559
wfd_depth_scale0.65_noddf_v1.5_10yrs   -0.096559
wfd_depth_scale0.80_noddf_v1.5_10yrs   -0.078400
dcr_nham2_ugr_v1.5_10yrs               -0.078400
short_exp_5ns_5expt_v1.5_10yrs         -0.078400
footprint_bluer_footprintv1.5_10yrs    -0.078400
footprint_gp_smoothv1.5_10yrs          -0.078400
var_expt_v1.5_10yrs                    -0.078400
dcr_nham1_ug_v1.5_10

In [99]:
# PLOT SETTINGS: Combine 1.6 False, include unrealistic True
annotations = {
    'footprint_bluer_footprintv1.5_10yrs':('sn', 0),
    'footprint_newBv1.5_10yrs':('sn', 0),
    'bulges_bs_v1.5_10yrs':('sn', 0),
    'footprint_big_sky_dustv1.5_10yrs':('sn', 0.08)
}

dropped_rel, metrics_original, misc_metrics = select_simulations(combine_1_6=False, 
                                                                 include_unrealistic=True,
                                                                 include_outstanding=False)


all_sims_df.loc[dropped_rel.index, 'area'] = 1
x_axis = 'static_Y10 effective survey area'
if x_axis not in misc_metrics.columns:
    misc_metrics[x_axis] = metrics_original[x_axis]
probes = ['fom', 'wl', 'sl', 'sn', 'pz']
themes = {'fom':0, 'lss':1, 'wl':1, 'sl':1, 'sn':0, 'kn':1, 'pz':1, 'snz':1}
plot_tools.comparison_plot(dropped_rel, metrics_display_names, misc_metrics, x_axis, 
                           probes=probes, themes=themes, ylabel=read_metrics.ylabel, 
                           xlabel='Effective area (deg$^2$)', savefig_path=fig_dir+'comparison_area',
                           save_plot=save_plot,
                           smoothing='binning', expand_ylim_by=[2.5, 1.2],
                           annotations=annotations)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Annotations: 1-\texttt{footprint\_bluer\_footprintv1.5\_10yrs}, 2-\texttt{footprint\_newBv1.5\_10yrs}, 3-\texttt{bulges\_bs\_v1.5\_10yrs}, 4-\texttt{footprint\_big\_sky\_dustv1.5\_10yrs}


# Depth

In [80]:
annotations = {
    'footprint_newAv1.5_10yrs':('sn', 0),
    'bulges_cadence_i_heavy_v1.5_10yrs':('fom', 0.08),
    'wfd_depth_scale0.99_v1.5_10yrs':('sn', 0.08),
    'wfd_depth_scale0.99_noddf_v1.5_10yrs':('sn', 0.08)
}
dropped_rel, metrics_original, misc_metrics = select_simulations(combine_1_6=False, 
                                                                 include_unrealistic=True,
                                                                 include_outstanding=False)


all_sims_df.loc[dropped_rel.index, 'depth'] = 1
x_axis = 'static_median Y10 $i$-band coadded depth in effective survey area'
if x_axis not in misc_metrics.columns:
    misc_metrics[x_axis] = metrics_original[x_axis]
probes = ['fom',  'sn', 'kn', 'sl', 'pz']
themes = {'fom':0, 'lss':1, 'wl':1, 'sn':0, 'kn':1, 'sl':1, 'pz':1, 'snz':1}
plot_tools.comparison_plot(dropped_rel, metrics_display_names, misc_metrics, x_axis, 
                           probes=probes, themes=themes, ylabel=read_metrics.ylabel, 
                           xlabel='Median i-band depth', savefig_path=fig_dir + 'comparison_depth',
                           save_plot=save_plot,
                           smoothing='binning', expand_ylim_by=[2.0, 1.8],
                           annotations=annotations)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Annotations: 1-\texttt{footprint\_newAv1.5\_10yrs}, 2-\texttt{bulges\_cadence\_i\_heavy\_v1.5\_10yrs}, 3-\texttt{wfd\_depth\_scale0.99\_v1.5\_10yrs}, 4-\texttt{wfd\_depth\_scale0.99\_noddf\_v1.5\_10yrs}


# Combined metric

In [81]:
# PLOT SETTINGS: combine_1.6 True, include_unrealistic False
def calculate_mega_metric(metrics_rel):
#     frac_static_tot = 0.68
#     frac_sn_tot = 0.32
    
    frac_static_tot = 0.5
    frac_sn_tot = 0.5
    
#     frac_static_statistical = 0.62
#     frac_static_systematic = 0.38
    
    frac_static_statistical = 1
    frac_static_systematic = 0
    
    frac_static_systematic_redshift = 0.5
    frac_static_systematic_psf = 0.5
    
    pz = metrics_rel[plot_tools.probes_translations['pz']]
    wl = metrics_rel[plot_tools.probes_translations['wl']]
    fom = metrics_rel[plot_tools.probes_translations['fom']]
    sn = metrics_rel[plot_tools.probes_translations['sn']]
    
    
    static_systematic = frac_static_systematic_redshift*pz + frac_static_systematic_psf*wl
    static_total = frac_static_systematic*static_systematic + frac_static_statistical*fom
    
    sn_total = sn
    
    return frac_static_tot*static_total + frac_sn_tot*sn_total

In [82]:
show_footprint_8 = False

dropped_rel, metrics_original, misc_metrics = select_simulations(combine_1_6=True, 
                                                                 include_unrealistic=False,
                                                                 include_outstanding=False,
                                                                remove_nexp2=True)

# Note there are slight variations in the sims used now vs. what was in the table originally but I think I 
# just forgot to update the table because otherwise the figure would look very different.

mega_metric = calculate_mega_metric(dropped_rel)
dropped_rel['mega'] = mega_metric

X = np.array([metrics_original.loc[dropped_rel.index, 'static_Y10 effective survey area'],
              #metrics_original.loc[dropped_rel.index, 'static_median Y10 $i$-band coadded depth in effective survey area'],
#             metrics_original.loc[dropped_rel.index, 'wl_systematics'],  
              misc_metrics.loc[dropped_rel.index, 'fONv MedianNvis fO All visits HealpixSlicer']
             ]).T
Z = np.array(dropped_rel.mega)
# Z = np.array(dropped_rel['kn_Pop_kNe_counts'])
# Z = np.array(dropped_rel['sl_num_lensed_SNe_Ia_with_good_time_delays_lensed_by_galaxies'])

xmin = X[:,0].min()
xmax = X[:,0].max()
ymin = X[:,1].min()
ymax = X[:,1].max()


extent = xmin, xmax, ymin, ymax

X[:,0] = (X[:,0] - xmin)/(xmax-xmin)
X[:,1] = (X[:,1] - ymin)/(ymax-ymin)



labels = [
         'ss_heavy_v1.6_10yrs',
        'footprint_newAv1.5_10yrs',
         'combo_dust_v1.6_10yrs',
         'footprint_big_sky_dustv1.5_10yrs',
         'baseline_v1.5_10yrs'
         ]

if show_footprint_8:
    labels += ['footprint_8_v1.710yrs',
    'baseline_nexp2_v1.7_10yrs']

xlabs = []
ylabs = []

# xshift = [0.005]*len(labels)

xshift = [-1000]*len(labels)
xshift[0] = 100
xshift[1] = -800
# xshift[2] = -400
# xshift[3] = -500
xshift[2] = -600
xshift[3] = -1200
xshift[4] = 100


yshift = [0]*len(labels)
yshift[1] = 15


if show_footprint_8:
    xshift[5] = -400
    xshift[6] = 100


descriptions = pd.read_excel('bar_graph_data.xlsx', sheet_name='descriptions', index_col=0)

if show_footprint_8:
    descriptions.loc['baseline_v1.4_10yrs.db', 'short'] = 'Baseline v1.5'
    descriptions.loc['footprint_big_sky_dustv1.4_10yrs.db', 'short'] = 'FootprintBigSky'
    descriptions.loc['baseline_nexp2_v1.7_10yrs.db'] = ['Baseline v1.7', 'Baseline v1.7']
    descriptions.loc['footprint_8_v1.710yrs.db'] = ['Footprint8', 'Footprint8']
    


for i in range(len(labels)):
    l = labels[i]
    xlab = metrics_original.loc[l, 'static_Y10 effective survey area']
#         xlab = metrics_original.loc[l, 'static_median Y10 $i$-band coadded depth in effective survey area']
#         ylab = metrics_original.loc[l, 'wl_systematics']
    ylab = misc_metrics.loc[l, 'fONv MedianNvis fO All visits HealpixSlicer']

#     xlabs.append((xlab-xmin)/(xmax-xmin))
#     ylabs.append((ylab-ymin)/(ymax-ymin))
    xlabs.append(xlab)
    ylabs.append(ylab)

    all_sims_df.loc[dropped_rel.index, 'mega_metric'] = 1
plt.close('all')
x = np.linspace(X[:,0].min(), X[:,0].max(), 200)
y = np.linspace(X[:,1].min(), X[:,1].max(), 200)
xgrid, ygrid = np.meshgrid(x, y)
zgrid = griddata(X, Z, (xgrid, ygrid), method='nearest')

plt.figure()

for i in range(len(labels)):
    plt.plot(xlabs[i], ylabs[i], 'ok', markeredgecolor='w')
#         this_text = labels[i]
#         if '.db' in this_text:
#             this_text = this_text[:-3]
#         if '_10yrs' in this_text:
#             this_text = this_text[:-6]
    ind = labels[i]+'.db'
    if '1.5' in ind:
        ind = ind.replace('1.5', '1.4')
    this_text = descriptions.loc[ind, 'short']
    plt.text(xlabs[i]+xshift[i], ylabs[i]+yshift[i], this_text, usetex=False, 
             fontsize=8,bbox=dict(facecolor='white', alpha=0.5))

#     if show_footprint_8:
#         plt.text(metrics_original.loc['footprint_8_v1.710yrs', 'static_Y10 effective survey area'], 26.28, '*', fontsize=30)
im1=plt.imshow(zgrid, origin='lower', extent=extent, aspect='auto', cmap='magma')
l1, l2 = 4,2
l3, l4 = 6,5
if show_footprint_8:
    plt.plot([xlabs[l1], xlabs[l2]], [ylabs[l1], ylabs[l2]], 'w-')
#     plt.plot([xlabs[l3], xlabs[l4]], [ylabs[l3], ylabs[l4]], 'w--')
plt.plot([])
cbar = plt.colorbar()
cbar.set_label('(metric-baseline)/baseline', rotation=-90, labelpad=20)
# cbar.set_label('KN (relative to baseline)', rotation=-90, labelpad=20)

plt.xlabel('Effective area (deg$^2$)')
#     plt.xlabel('Median $i$-band depth')
plt.ylabel('Median number of visits per pointing')
plt.tight_layout()
plt.savefig(fig_dir+'mega_metric')
# plt.savefig(fig_dir+'kn_combined.png')
# plt.savefig(fig_dir+'sl_combined.png')
#     plt.savefig(fig_dir+'nvists_depth_combined.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Filter distribution

In [83]:
metrics_rel = metrics_rel_1_5.copy()

inds = []
for i in metrics_rel.index:
    if 'filterdist' in i:
        inds.append(i)
# Put baseline first
filters_baseline = 'filterdist_indx2_v1.5_10yrs'
inds[1] = inds[0]
inds[0] = filters_baseline

filters_only = metrics_original.loc[inds]

filters_only = (filters_only-filters_only.loc[filters_baseline])/filters_only.loc[filters_baseline]


plt.figure()

labels = ['Baseline', 'Uniform','g heavy','u heavy', 'z and y heavy', 'i heavy', 'Bluer', 'Redder']

probes = ['wl', 'sn', 'kn', 'sl', 'pz', 'snz']
x = np.arange(len(filters_only))

plt.plot(x, [0]*len(x), 'k--', alpha=0.5)

for p in probes:
    metric_name = plot_tools.probes_translations[p]
    y = filters_only[metric_name]
    plt.plot(x, y, label=metrics_display_names[metric_name], 
             color=plot_tools.colors[p], marker=plot_tools.markers[p])
    
plt.xticks(x, labels)
plt.xlabel('Filter distribution')
plt.ylabel("(metric-filterdist baseline)/filterdist baseline")

plt.legend()
plt.xlim([0, len(x)-1])
plt.ylim([-0.3, 0.65])
plt.tight_layout()


all_sims_df.loc[inds, 'filter_dist'] = 1
plt.savefig(fig_dir + 'comparison_filter_dist')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Bar graph - Same filter visits

In [30]:
reload(plot_tools)

<module 'plot_tools' from '/home/michelle/Project/OSTF/journal_plots/plot_tools.py'>

In [88]:
metrics_rel_samefilt, metrics_display_names, baseline_values_samefilt = read_metrics.read_metrics('metrics_v1_5.csv', baseline='baseline_v1.5_10yrs', 
                                                                                                  dropna=False, 
                                                                                                  add_samefilt=True)

this_df = metrics_rel_samefilt.loc[['baseline_samefilt_v1.5_10yrs.db']]
plot_tools.single_bar_graph(this_df, metrics_display_names, ylabel=read_metrics.ylabel,
                           savefig_path=fig_dir+'comparison_samefilt', save_plot=True,
                           probes=['wl', 'lss', 'fom', 'sl', 'sn'], figsize=figsize_fullwidth)
all_sims_df.loc['baseline_samefilt_v1.5_10yrs', 'samefilt'] = 1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Bar graph - nexp

In [86]:
reload(plot_tools)

<module 'plot_tools' from '/home/michelle/Project/OSTF/journal_plots/plot_tools.py'>

In [89]:

this_df = metrics_rel_1_7.loc[['baseline_nexp1_v1.7_10yrs']]
plot_tools.single_bar_graph(this_df, metrics_display_names, ylabel=read_metrics.ylabel,
                       savefig_path=fig_dir+'comparison_1nexp', save_plot=True,
                       probes=['lss','fom', 'wl','pz','sl','kn','sn'], figsize=figsize_fullwidth,
                       expand_ylim_by=[1,1.1])
all_sims_df['nexp'] = [0]*len(all_sims_df)
inds_7 = []
for i in all_sims_df.index:
    if '1.7' in i:
        inds_7.append(i)
all_sims_df.loc[inds_7, 'nexp'] = 1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Internight gap

In [90]:
# PLOT SETTINGS: combine 1.6 False, include unrealistic False
annotations = {
    'dcr_nham2_ugr_v1.5_10yrs':('sl', 0.08),
    'baseline_2snaps_v1.5_10yrs':('sn', 0),
    'footprint_newAv1.5_10yrs':('sn', 0)
}

dropped_rel, metrics_original, misc_metrics = select_simulations(combine_1_6=False, 
                                                                 include_unrealistic=False,
                                                                 include_outstanding=False)
all_sims_df.loc[dropped_rel.index, 'internight_gap'] = 1
# x_axis = to_include[-1]
x_axis = 'Median Median Inter-Night Gap all bands HealpixSlicer'
# x_axis = 'Median Max Inter-Night Gap all bands HealpixSlicer'
if x_axis not in misc_metrics.columns:
    misc_metrics[x_axis] = metrics_original[x_axis]
probes = ['fom', 'wl', 'sl', 'sn', 'pz']
themes = {'fom':0, 'lss':1, 'wl':1, 'sl':1, 'sn':0, 'kn':1, 'pz':1, 'snz':1}

plot_tools.comparison_plot(dropped_rel, metrics_display_names, misc_metrics, x_axis, 
                           probes=probes, themes=themes, ylabel=read_metrics.ylabel, 
                           xlabel='Median inter-night gap all bands', savefig_path=fig_dir+'comparison_internight_gap',
                           save_plot=True,
                           smoothing='binning', expand_ylim_by=[1.4, 1.2],
                           annotations=annotations)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Annotations: 1-\texttt{dcr\_nham2\_ugr\_v1.5\_10yrs}, 2-\texttt{baseline\_2snaps\_v1.5\_10yrs}, 3-\texttt{footprint\_newAv1.5\_10yrs}


# Internight gap per filter

In [91]:
import george
from scipy.optimize import minimize

dropped_rel, metrics_original, misc_metrics = select_simulations(combine_1_6=False, 
                                                                 include_unrealistic=True,
                                                                 include_outstanding=False)


bands = ['u','g','r','i','z','y']
cols = ['C4', 'C2', 'C3', 'C1', 'C5', 'C0']
normalise_x = False
# which_metric = 'kn_Pop_kNe_counts'
which_metric = 'sn_normal_nsn_tot'
# which_metric = 'static_median Y10 $i$-band coadded depth in effective survey area'
# which_metric = 'pz_y10_zlow_stdd'
# which_metric = 'sl_num_lensed_SNe_Ia_with_good_time_delays_lensed_by_galaxies'
# which_metric = 'fONv MedianNvis fO All visits HealpixSlicer'



inds = []
for i in dropped_rel.index:
    if 'short_exp' not in i and 'footprint_bluer' not in i:
        inds.append(i)
        
col = 'fONv MedianNvis fO All visits HealpixSlicer'
inds = misc_metrics.loc[inds][(misc_metrics.loc[inds, col]<=1000)&(misc_metrics.loc[inds, col]>=900)].index

k = 0
plt.figure()
for b in bands:
    x_axis = 'Median Median Inter-Night Gap %s band HealpixSlicer' %b
    if x_axis not in misc_metrics:
        misc_metrics = pd.merge(misc_metrics, maf_metrics5[x_axis], how='inner', left_index=True, right_index=True)
    
    x = misc_metrics.loc[inds, x_axis]
    if normalise_x:
        x = (x-x.min())/(x.max()-x.min())
    y = dropped_rel.loc[inds, which_metric]
    
    
#     y = misc_metrics.loc[inds, which_metric]
    #x = x/misc_metrics.loc[inds, 'Median Median Inter-Night Gap all bands HealpixSlicer']
#     x = x*misc_metrics.loc[inds, 'fONv MedianNvis fO All visits HealpixSlicer']

    kernel = george.kernels.ExpSquaredKernel((x.max()-x.min()))

    gp = george.GP(kernel, fit_mean=True, fit_white_noise=True)
    gp.compute(x, y.std())

    def neg_ln_like(p):
            gp.set_parameter_vector(p)
            return -gp.log_likelihood(y)

    def grad_neg_ln_like(p):
        gp.set_parameter_vector(p)
        return -gp.grad_log_likelihood(y)

    result = minimize(neg_ln_like, gp.get_parameter_vector(),
                              jac=grad_neg_ln_like, method="L-BFGS-B",
                              tol = 1e-5)
    gp.set_parameter_vector(result.x)

    pred_x = np.linspace(x.min(), x.max(), 100)
    pred_y, y_var = gp.predict(y, pred_x, return_var=True)
    other_pred_y = gp.predict(y, x, return_cov=False)
    
    new_x = pred_x
    new_y = pred_y
    
    y_err = np.sqrt(y_var+(y-other_pred_y).std()**2)
    
    baseline_x = misc_metrics.loc['baseline_v1.5_10yrs',x_axis]
    baseline_y = 0

    if not normalise_x:
        plt.fill_between(new_x, new_y-y_err, new_y+y_err, color=cols[k], alpha=0.3)
    plt.plot(new_x, new_y, color=cols[k], label=b)
    
    plt.plot(x, y, '.', markersize=1, color=cols[k]
        )
    plt.plot(baseline_x, baseline_y, '*', markersize=8, 
             markerfacecolor=cols[k], markeredgecolor='k', markeredgewidth=0.3)
    plt.text(new_x.max()+0.1, new_y[new_x.argmax()]-0.04, '$\mathbf{%s}$' %b, color=cols[k], fontsize=12)
    k+=1
plt.xlabel('Median internight-gap (days)')
plt.ylabel('(metric-baseline)/baseline')
plt.title('Number of  SNe Ia with z$\\textless$zlim(med)')
plt.legend()
if not normalise_x:
    plt.xlim([4.9,26])
#     plt.ylim([-0.15, 0.05])
plt.tight_layout()

all_sims_df.loc[inds, 'internight_gap_per_filter'] = 1
if save_plot:
    plt.savefig(fig_dir+'internight_gap_per_filter')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Season length

In [92]:
annotations = {
    'footprint_newBv1.5_10yrs':('sn', 0),
    'wfd_depth_scale0.99_v1.5_10yrs':('sn', -0.08),
    'short_exp_5ns_5expt_v1.5_10yrs':('sn', 0.02),
    'dcr_nham1_ugri_v1.5_10yrs':('sl', 0.08)
}
# annotations = {}
dropped_rel, metrics_original, misc_metrics = select_simulations(combine_1_6=False, 
                                                                 include_unrealistic=True,
                                                                 include_outstanding=False)

all_sims_df.loc[dropped_rel.index, 'season'] = 1
x_axis = 'cumulative_season_length_[yr]'
if x_axis not in misc_metrics.columns:
    misc_metrics[x_axis] = metrics_original[x_axis]
probes = ['sl', 'sn', 'fom']
# probes = ['sl']
themes = {'sl':0, 'sn':1, 'kn':1, 'fom':1, 'pz':1, 'snz':1}
plot_tools.comparison_plot(dropped_rel, metrics_display_names, misc_metrics, 
                           x_axis, probes=probes, themes=themes, ylabel=read_metrics.ylabel, 
                           xlabel='Cumulative season length (years)', savefig_path=fig_dir+'comparison_season',
                           save_plot=save_plot,
                           smoothing='binning', expand_ylim_by=[1.3,1.1],
                           annotations=annotations)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Annotations: 1-\texttt{footprint\_newBv1.5\_10yrs}, 2-\texttt{wfd\_depth\_scale0.99\_v1.5\_10yrs}, 3-\texttt{short\_exp\_5ns\_5expt\_v1.5\_10yrs}, 4-\texttt{dcr\_nham1\_ugri\_v1.5\_10yrs}


# Weather

In [93]:
fl4 = 'metrics_v1_4.csv'
misc_metrics_fl4 = 'misc-metrics-v1_4.csv' 

misc_metrics4 = read_and_drop(misc_metrics_fl4)

if 'dbname' in misc_metrics4.index:
    misc_metrics4 = misc_metrics4.drop('dbname')
metrics_original4 = read_and_drop(fl4)
metrics_rel4, metrics_display_names4, baseline_values4 = read_metrics.read_metrics(fl4, 
                                                                                   baseline='baseline_v1.4_10yrs.db')

misc_metrics4 = misc_metrics4.loc[metrics_original4.index]

weather_runs = [i for i in metrics_rel4.index if 'weather' in i]
metrics_weather = metrics_rel4.loc[weather_runs]
new_inds = metrics_weather.index[[0,1,3,2]]
metrics_weather = metrics_weather.loc[new_inds]
drop_db_suffix(metrics_weather)

all_sims_df.loc[metrics_weather.index, 'weather'] = 1

plt.figure()

probes = ['fom', 'lss', 'sn', 'kn', 'sl']
x = np.arange(len(metrics_weather.index))
for p in probes:
    metric_name = plot_tools.probes_translations[p]
    y = metrics_weather[metric_name]
    plt.plot(x, y, label=metrics_display_names[metric_name], 
             color=plot_tools.colors[p], marker=plot_tools.markers[p])
    
# y = metrics_weather['static_stddev of Y10 $i$-band coadded depth in effective survey area']
# plt.plot(x, y, label='Depth', color='C7')
# y = metrics_weather['static_Y10 effective survey area']
# plt.plot(x, y, label='Area', color='C8')
plt.xticks(x, ['Realistic', 'v1.3-like', 'Optimistic', 'Optimistic - \nno downtime'])
plt.xlabel('Weather simulation type')
plt.ylabel(read_metrics.ylabel)
plt.tight_layout()
plt.legend()
plt.savefig(fig_dir + 'comparison_weather.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Write simulations table

In [213]:
# existing_sims_df = pd.read_csv('simulations_table.csv', index_col=0)
# (all_sims_df == existing_sims_df).sum()
# There are differences in the mega metric only (2021/11/09)

In [94]:
all_sims_df.to_csv('simulations_table.csv')
overwrite_table = True

In [97]:
paper_figures

{'area': '\\ref{fig:comparison_area}',
 'depth': '\\ref{fig:comparison_depth}',
 'filter_dist': '\\ref{fig:comparison_filter_dist}',
 'internight_gap': '\\ref{fig:comparison_cadence}',
 'samefilt': '\\ref{fig:comparison_samefilt}',
 'season': '\\ref{fig:comparison_season}',
 'weather': '\\ref{fig:comparison_weather}',
 'internight_gap_per_filter': '\\ref{fig:internight_gap_filters}',
 'mega_metric': '\\ref{fig:mega_metric}',
 'nexp': '\\ref{fig:comparison_nexp}'}

In [98]:
if overwrite_table:
    all_sims_df = pd.read_csv('simulations_table.csv', index_col=0)
    inds = []
    for i in all_sims_df.index:
        if 'v1.6' in i and 'nexp2' in i:
            inds.append(i)
    all_sims_df = all_sims_df.drop(inds)
    
#     inds = np.array(list(paper_figures.values())).argsort()
#     keys = np.array(list(paper_figures.keys()))[inds]
    # Order taken from the paper
    keys = ['area', 'depth', 'mega_metric', 'filter_dist', 'samefilt', 'nexp', 'internight_gap', 
           'internight_gap_per_filter', 'season', 'weather']

    tab = open('simulations_table.tbl', 'w')

    # tab.write('\\hline\n')
    tab.write('Simulation name & Comparison baseline & Included in which figures\\\\ \n')
    tab.write('\\hline\n')

    for idx in all_sims_df.index:
        if idx == 'baseline_v1.5_10yrs':
            baseline_1_5 = True
        else:
            baseline_1_5 = False

        if idx == 'baseline_nexp1_v1.6_10yrs':
            baseline_1_6 = True
        else:
            baseline_1_6 = False

        which_figs = ''
        not_run = all_sims_df.loc[idx, all_sims_df.columns[1:]].sum() == 0
        if not baseline_1_5 and not baseline_1_6 and not_run:
            which_figs = 'Excluded, '
        else:
            for col in keys:
                if baseline_1_5 and col != 'filter_dist':
                    override = True
                elif baseline_1_6 and col == 'mega_metric':
                    override = True
                else:
                    override = False

                if col in all_sims_df.columns:
                    if all_sims_df.loc[idx, col] == 1  or override:
                        fig = str(paper_figures[col])
                        which_figs += fig + ', '

        which_figs = which_figs[:-2]

        sim_name = '\\texttt{%s}' %idx
        sim_name = sim_name.replace('_', '\\_')

        if '1.4' in idx:
            which_baseline = 'weather_0.3_v1.4_10yrs'
        elif '1.5' in idx:
            which_baseline = 'baseline_v1.5_10yrs'
        elif '1.6' in idx:
            which_baseline = 'baseline_nexp1_v1.6_10yrs'
        else:
            which_baseline = 'baseline_nexp2_v1.7_10yrs'
            
        which_baseline = '\\texttt{%s}' %which_baseline
        which_baseline = which_baseline.replace('_', '\\_')
        
        tab.write('%s & %s & %s \\\\ \n' %(sim_name, which_baseline, which_figs))
    tab.write('\\hline\n')

    tab.close()